# pytorchvideo UFC101, pytorchvideo X3D pretrain/scratch

pytorchvideonのdatasetを使ってUFC101を読み込み，pytorchvideoのx3dモデルをfine-tuningしてみる．
UFC101はあらかじめダウンロードして展開済みであるとする．

- https://pytorchvideo.readthedocs.io/en/latest/api/data/data.html#ucf101

- https://pytorch.org/hub/facebookresearch_pytorchvideo_x3d/



## ダウンロードできないというエラー

torchvisionをimportした後ではエラーが発生する（ImportError: cannot import name ***）

- https://github.com/pytorch/hub/issues/46


## 対応策

import torch直後に（import torchvisionをしない状態で）torch.hub.loadして，キャッシュに残しておく

こうすると，以降はキャッシュ（~/.cache/torch/hub/checkpoints/）が使われるのでエラーは発生しない

In [1]:
from comet_ml import Experiment
import torch
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_xs', pretrained=True)
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_s', pretrained=True)
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_m', pretrained=True)


ValueError: Cannot find master in https://github.com/facebookresearch/pytorchvideo. If it's a commit from a forked repo, please call hub.load() with forked repo directly.

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data import DistributedSampler, RandomSampler


from torchvision import transforms


from pytorchvideo.models import x3d
from pytorchvideo.data import Ucf101, RandomClipSampler, UniformClipSampler


from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
)


#import torchinfo

from tqdm.notebook import tqdm
import itertools
import os
import pickle

argparseを真似たパラメータ設定．
- rootで指定したディレクトリには，101クラスのサブディレクトリがあること
- annotation_pathにはtrainlist0{1,2,3}.txtなどがあること

In [5]:
class Args:
    def __init__(self):
        self.metadata_path = '/mnt/NAS-TVS872XT/dataset/UFC101/'
        self.root = self.metadata_path + 'video/'
        self.annotation_path = self.metadata_path + 'ucfTrainTestlist/'
        self.frames_per_clip = 16
        self.step_between_clips = 16
        self.model = 'x3d_m'
        self.batch_size = 16
        self.num_workers = 24

        self.clip_duration = 16/25  # 25FPSを想定して16枚
        self.video_num_subsampled = 16  # 16枚抜き出す

args = Args()

transformの定義．
- UniformTemporalSubsampleで固定枚数をサンプルする
 - datasetのclip_samplerには，秒単位でしか与えられないようなので，fpsが異なる動画ではサンプルされる枚数も変わってくる．そのためここで取得するフレーム数を揃える（もっといい方法はないのか？）
- UCF101を読み込むとfloat32だが値は0-255，255で割ってfloatにする．
- X3D-Mを想定して，短い方を256画素程度に合わせてから，画像を224x224にリサイズする．
  - RandomShortSideScaleなら厳密には256にならない
  - ShortSideScaleなら256になる

バッチはdict形式なので，video, label, audioなどのそれぞれにtransformが設定できる
- ApplyTransformToKeyでkeyを指定して，video用のtransformを設定
- UCF101のラベルファイル（trainlist01.txtなど）には1から101までのラベルが付いているが，それがそのまま使われてしまうので（なぜだ．．．），このままではエラーが（不定期に）発生する．ラベルの値をtransformでから100にしておく
- audioは使わないのでRemoveKeyで除去

In [6]:
train_transform = Compose([
    ApplyTransformToKey(
        key="video",
        transform=Compose([
                UniformTemporalSubsample(args.video_num_subsampled),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                ## 以下デバッグ用
                # transforms.Lambda(lambda x: [
                #     x, 
                #     print(type(x)),
                #     print(x.dtype),
                #     print(x.max()),
                #     print(x.min()),
                #     print(x.mean()),
                #     ]),
                # transforms.Lambda(lambda x: x[0]),
                RandomShortSideScale(min_size=256, max_size=320,),
                RandomCrop(224),
                RandomHorizontalFlip(),
        ]),
    ),
    ApplyTransformToKey(
        key="label",
        # ラベルが1から101になっているので，1を引いておく
        transform=transforms.Lambda(lambda x: x - 1),
    ),
    RemoveKey("audio"),
])

val_transform = Compose([
    ApplyTransformToKey(
        key="video",
        transform=Compose([
                UniformTemporalSubsample(args.video_num_subsampled),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                ShortSideScale(256),
                CenterCrop(224),
        ]),
    ),
    ApplyTransformToKey(
        key="label",
        # ラベルが1から101になっているので，1を引いておく
        transform=transforms.Lambda(lambda x: x - 1),
    ),
    RemoveKey("audio"),
])



In [15]:
root_UCF101 = '/mnt/NAS-TVS872XT/dataset/UCF101/'

train_set = Ucf101(
    data_path=root_UCF101 + 'ucfTrainTestlist/trainlist01.txt',  # ラベルが1から101になっているので，transformで1を引いている
    video_path_prefix=root_UCF101 + 'video/',
    clip_sampler=RandomClipSampler(clip_duration=args.clip_duration),
    video_sampler=RandomSampler,
    decode_audio=False,
    transform=train_transform,
    )
val_set = Ucf101(
    data_path=root_UCF101 + 'ucfTrainTestlist/testlist.txt',
    video_path_prefix=root_UCF101 + 'video/',
    clip_sampler=RandomClipSampler(clip_duration=args.clip_duration),
    video_sampler=RandomSampler,
    decode_audio=False,
    transform=val_transform,
    )

num_classes = 101

In [16]:
train_set.num_videos

9537

In [17]:
val_set.num_videos

3783

In [18]:
# https://github.com/facebookresearch/pytorchvideo/blob/ef2d3a96bb939b12aa0f21fb467d2175b0f05e9f/tutorials/video_classification_example/train.py#L343
class LimitDataset(torch.utils.data.Dataset):
    """
    To ensure a constant number of samples are retrieved from the dataset we use this
    LimitDataset wrapper. This is necessary because several of the underlying videos
    may be corrupted while fetching or decoding, however, we always want the same
    number of steps per epoch.
    """

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
        self.dataset_iter = itertools.chain.from_iterable(
            itertools.repeat(iter(dataset), 2)
        )

    def __getitem__(self, index):
        return next(self.dataset_iter)

    def __len__(self):
        return self.dataset.num_videos

In [19]:
train_loader = DataLoader(LimitDataset(train_set),
                            batch_size=args.batch_size,
                            drop_last=True,
                            num_workers=args.num_workers)
val_loader = DataLoader(LimitDataset(val_set),
                            batch_size=args.batch_size,
                            drop_last=True,
                            num_workers=args.num_workers)


データローダのlenを確認．
- trainlist01.txtには9537行あるので「サンプル数＝ビデオ数」
- バッチサイズで割るとtrain_loaderのlengthになる

In [20]:
len(train_loader), train_set.num_videos, train_set.num_videos / args.batch_size

(596, 9537, 596.0625)

data loaderの挙動を確認．
- バッチはdictでやってくるので，`batch['video']`と`batch['label']`で取り出す
- RandomClipSamplerならランダムなラベルが得られている．

In [22]:
for i, batch in enumerate(val_loader):
    if i == 0:
        print(batch.keys())
        print(batch['video'].shape)
    print(batch['label'].cpu().numpy())
    if i > 10:
        break

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])
torch.Size([16, 3, 16, 224, 224])
[34 84 60 34 32 21 92 68  1 23 55 23 41 51 81 58]
[97  5 44 22 77 28 13 94 78 54 19 96 77 52 36 69]
[ 26  83  23  58  16   4  82   1  91  56  73  84  31   0 100  31]
[54 20 74 48 31  9 54 71 31 74 15  7 32 95 25 18]
[ 6 61 97 91 26 12 38 20 63 90 88 52 46 57 98 78]
[73 63 32 40 96 84 44 33  8  3 38 11 75 42 25 78]
[59 70 25 60 46 56 40 14 29 62  4 92 14 30  8 89]
[92 58 54 64 89 82 42 49  8 10  8 62 77  9 56 30]
[ 8 11 56 90 26 45  9 79 83 65 28 37 22 16 20 93]
[ 7 19 53 95 70 32 98 66  5 75 26  8 42 50 15 58]
[87 94 13 43 43 62 74 73 35 50 45 48 52 65 67 43]
[ 52  89  51  41  91  53   9  58  97 100  75   6  78  13  24  14]


In [11]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# device = "cpu"

pytorchvideoのpretrained x3dモデルをダウンロード．
あとでsummaryを見れば分かるように，最終線形層は`model.blocks[5].proj`だからこれをnn.Linearに置き換える

- 注意：エラーが発生してダウンロードできない場合には，このnotebookの冒頭の注意書きを確認すること

In [12]:
# # X3D-M
# # https://github.com/facebookresearch/pytorchvideo/blob/master/pytorchvideo/models/x3d.py#L601
# model = x3d.create_x3d(
#     input_clip_length=16,
#     input_crop_size=224,
#     depth_factor=2.2,
#     model_num_class=101
# ).to(device)


model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_m', pretrained=True)

# fine-tuningするなら以下を実行．スクラッチで学習するなら，実行しない
do_fine_tune = True
if do_fine_tune:
    for param in model.parameters():
        param.requires_grad = False

model.blocks[5].proj = nn.Linear(model.blocks[5].proj.in_features, num_classes)
model = model.to(device)

# data parallelだと性能が落ちる（設定次第？）
# model = nn.DataParallel(model)

Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master


ランダムなデータを流し込んで出力されるかを確認する

In [15]:
data = torch.randn(2, 3, 16, 224, 224).to(device)

In [16]:
model(data)

tensor([[0.0108, 0.0107, 0.0102, 0.0087, 0.0105, 0.0091, 0.0093, 0.0095, 0.0092,
         0.0104, 0.0097, 0.0099, 0.0098, 0.0088, 0.0094, 0.0097, 0.0087, 0.0108,
         0.0105, 0.0103, 0.0094, 0.0111, 0.0100, 0.0106, 0.0089, 0.0093, 0.0096,
         0.0107, 0.0105, 0.0101, 0.0096, 0.0104, 0.0101, 0.0110, 0.0092, 0.0100,
         0.0094, 0.0097, 0.0112, 0.0095, 0.0092, 0.0105, 0.0099, 0.0095, 0.0101,
         0.0088, 0.0097, 0.0099, 0.0111, 0.0103, 0.0103, 0.0103, 0.0104, 0.0099,
         0.0105, 0.0099, 0.0091, 0.0097, 0.0096, 0.0103, 0.0100, 0.0098, 0.0109,
         0.0106, 0.0096, 0.0094, 0.0098, 0.0105, 0.0088, 0.0094, 0.0100, 0.0101,
         0.0102, 0.0109, 0.0104, 0.0094, 0.0103, 0.0099, 0.0091, 0.0104, 0.0107,
         0.0093, 0.0107, 0.0097, 0.0091, 0.0089, 0.0099, 0.0100, 0.0101, 0.0105,
         0.0093, 0.0106, 0.0099, 0.0089, 0.0094, 0.0097, 0.0103, 0.0100, 0.0091,
         0.0093, 0.0100],
        [0.0102, 0.0102, 0.0100, 0.0105, 0.0096, 0.0100, 0.0096, 0.0088, 0.0081,
  

summaryで中身を確認

In [17]:
# torchinfo.summary(
#     model,
#     (4, 3, 16, 224, 224),
#     depth=4,
#     col_names=["input_size",
#                "output_size"],
#     row_settings=("var_names",)
# )

便利関数を定義

In [13]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    https://github.com/machine-perception-robotics-group/attention_branch_network/blob/ced1d97303792ac6d56442571d71bb0572b3efd8/utils/misc.py#L59
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        if type(val) == torch.Tensor:
            val = val.item()
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def top1(outputs, targets):
    batch_size = outputs.size(0)
    _, predicted = outputs.max(1)
    return predicted.eq(targets).sum().item() / batch_size

In [14]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [15]:
hyper_params = {
    "num_classes": 101,
    "batch_size": args.batch_size,
    "num_epochs": 5,
    "learning_rate": 0.1
}

In [16]:
# !pip install comet_ml

Defaulting to user installation because normal site-packages is not writeable


In [18]:
from comet_ml import Experiment

experiment = Experiment(api_key="TawRAwNJiQjPaSMvBAwk4L4pF",
                        project_name="test0_comet", workspace="kazukiomi")
experiment.add_tag("pytorch")
experiment.log_parameters(hyper_params)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/kazukiomi/test-comet/e0e2ec4690a245f890eaa834d46af7c3
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 16
COMET INFO:     learning_rate : 0.1
COMET INFO:     num_classes   : 101
COMET INFO:     num_epochs    : 5
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (127.52 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     os packages              : 1
COMET INFO: ---------------------------
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET ERROR: Failed to report experiment status
COMET WARNING: As you are running in a Jupyter environ

In [19]:
num_epochs = 5

model = model.to(device)

with tqdm(range(num_epochs)) as pbar_epoch:
    for epoch in pbar_epoch:
        pbar_epoch.set_description("[Epoch %d]" % (epoch))


        with tqdm(enumerate(train_loader),
                  total=len(train_loader),
                  leave=True) as pbar_loss:

            train_loss = AverageMeter()
            train_acc = AverageMeter()
            model.train()

            for batch_idx, batch in pbar_loss:
                pbar_loss.set_description("[train]")

                inputs, targets = batch['video'].to(device), batch['label'].to(device)
                bs = inputs.size(0)  # current batch size, may vary at the end of the epoch

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss.update(loss, bs)
                train_acc.update(top1(outputs, targets), bs)

                experiment.log_metric("batch_acc", train_acc.val, step=batch_idx)

                pbar_loss.set_postfix_str(
                    ' | loss={:6.04f} , top1={:6.04f}'
                    ' | loss={:6.04f} , top1={:6.04f}'
                    ''.format(
                    train_loss.avg, train_acc.avg,
                    train_loss.val, train_acc.val,
                ))
            
            experiment.log_metric("epoch_acc", train_acc.avg, step=epoch)



fine-tuningなので速い．
- 4GPUでおよそ4.5it/s，1エポック約2分
- 1GPUでおよそ5it/s，1エポック約3分（596 iterations）

スクラッチで学習するなら
- 4GPUでおよそ2.6it/s，1エポック約4分
- 1GPUでおよそ1.8it/s，1エポック約5.5分（596 iterations）


以下の設定
- video_num_subsampled = 16
- batch_size = 16
- num_workers = 24

In [21]:
import os
os.cpu_count()

36